In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [2]:
df_monthly = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios.CSV', header=6)
df_monthly = df_monthly.rename(columns={'Unnamed: 0': 'Date'})

df_48ind = df_monthly.iloc[:1171].copy()
df_48ind['Date'] = pd.to_datetime(df_48ind['Date'], format='%Y%m')
df_48ind.set_index('Date', inplace=True)
df_48ind = df_48ind.apply(pd.to_numeric, errors='coerce')
df_48ind.replace(-99.99, np.nan, inplace = True)
df_48ind.replace(-999, np.nan, inplace = True)
df_48ind.dropna(inplace = True)

df_numfirm = df_monthly.iloc[2564-20:3735-20].copy()
df_numfirm['Date'] = pd.to_datetime(df_numfirm['Date'], format='%Y%m')
df_numfirm.set_index('Date', inplace=True)
df_numfirm = df_numfirm.apply(pd.to_numeric, errors='coerce')

df_avgsize = df_monthly.iloc[3739-22:4910-22].copy()
df_avgsize['Date'] = pd.to_datetime(df_avgsize['Date'], format='%Y%m')
df_avgsize.set_index('Date', inplace=True)
df_avgsize = df_avgsize.apply(pd.to_numeric, errors='coerce')


df_MC = df_numfirm.multiply(df_avgsize, axis=0)
df_MC = df_MC.loc[df_48ind.index]


df_BtoM = df_monthly.iloc[4890:4988].copy()
df_BtoM = df_BtoM.apply(pd.to_numeric, errors='coerce')

df_BtoM = df_BtoM.loc[df_BtoM.index.repeat(12)].reset_index(drop=True)

df_BtoM['Date'] = pd.to_datetime(df_BtoM['Date'], format='%Y')

df_BtoM = df_BtoM.drop('Date', axis=1)

df_BtoM.replace(-99.99, np.nan, inplace = True)
df_BtoM.replace(-999, np.nan, inplace = True)
df_BtoM.dropna(inplace = True)

df_BtoM = pd.DataFrame(data = df_BtoM.iloc[5:].values, index = df_48ind.index, columns = df_48ind.columns)


df_mom = df_48ind.rolling(window=12).mean()

df_mom.replace(-99.99, np.nan, inplace = True)
df_mom.replace(-999, np.nan, inplace = True)
df_mom.dropna(inplace = True)

df_48ind = df_48ind.loc[df_mom.index]
df_MC = df_MC.loc[df_mom.index]
df_BtoM = df_BtoM.loc[df_mom.index]

df_ret_shift = df_48ind.shift(-1)

df_FF_5factors = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/F-F_Research_Data_5_Factors_2x3.CSV', header=2)
df_FF_5factors = df_FF_5factors.rename(columns={'Unnamed: 0': 'Date'})
df_FF_5 = df_FF_5factors.iloc[:727].copy()
df_FF_5['Date'] = pd.to_datetime(df_FF_5['Date'], format='%Y%m')
df_FF_5.set_index('Date', inplace=True)
df_FF_5 = df_FF_5.apply(pd.to_numeric, errors='coerce')
df_FF_5 = df_FF_5.loc[df_mom.index]

df_daily = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios_Daily.csv', header=5)
df_daily = df_daily.rename(columns={'Unnamed: 0': 'Date'})

df_daily_ret = df_daily.iloc[:25670].copy()
df_daily_ret['Date'] = pd.to_datetime(df_daily_ret['Date'], format='%Y%m%d')
df_daily_ret.set_index('Date', inplace=True)
df_daily_ret = df_daily_ret.apply(pd.to_numeric, errors='coerce')
df_daily_ret.replace(-99.99, np.nan, inplace = True)
df_daily_ret.replace(-999, np.nan, inplace = True)
df_daily_ret.dropna(inplace = True)

df_daily_ret_reshaped = df_daily_ret.iloc[231:].copy()

df_FF_3 = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/F-F_Research_Data_Factors_daily.CSV', header=3)
df_FF_3 = df_FF_3.rename(columns={'Unnamed: 0': 'Date'})
df_FF_daily = df_FF_3.iloc[:25670].copy()
df_FF_daily['Date'] = pd.to_datetime(df_FF_daily['Date'], format='%Y%m%d')
df_FF_daily.set_index('Date', inplace=True)
df_FF_daily = df_FF_daily.apply(pd.to_numeric, errors='coerce')
df_FF_daily.replace(-99.99, np.nan, inplace = True)
df_FF_daily.replace(-999, np.nan, inplace = True)
df_FF_daily.dropna(inplace = True)
df_FF_daily = df_FF_daily.loc[df_daily_ret.index]

rf = df_FF_daily['RF'].mean()

########## boucle pour toutes les colonnes, avec une date de début et de fin, et pour chaque mois


def calculate_betas(start_date, end_date, df_daily_ret_reshaped, df_FF_daily):
    # Sélectionner les données pour la plage de dates spécifiée
    df_daily_ret_selected = df_daily_ret_reshaped.loc[start_date:end_date]
    df_FF_daily_selected = df_FF_daily.loc[start_date:end_date]

    # Maintenant, vous pouvez utiliser df_daily_ret_selected et df_FF_daily_selected pour votre régression
    Rft = df_FF_daily_selected['RF']
    Rm_t = df_FF_daily_selected['Mkt-RF']

    betas = []  # Create an empty list to store the betas

    for column in df_daily_ret_selected.columns:
        Ri_t = df_daily_ret_selected[column]

        Y = Ri_t - Rft
        X = Rm_t

        X = sm.add_constant(X)

        model = sm.OLS(Y, X)
        results = model.fit()

        betas.append(results.params[1])  # Add the betas to the list

    # Convert the list of betas to a numpy array
    betas = np.array(betas)

    return betas

# Get the first and last date in the data
first_date = df_daily_ret_reshaped.index.min()
last_date = df_daily_ret_reshaped.index.max()

# Create a date range for each month in the data
date_range = pd.date_range(start=first_date, end=last_date, freq='M')

# Calculate the betas for each month
monthly_betas = {}
for date in date_range:
    start_date = date - pd.DateOffset(months=12)
    end_date = date
    betas = calculate_betas(start_date, end_date, df_daily_ret, df_FF_daily)
    monthly_betas[date] = betas
    
    # Convert the dictionary to a DataFrame
df_monthly_betas = pd.DataFrame(monthly_betas.items(), columns=['Date', 'Betas'])

# Convert the Betas column from numpy arrays to lists
df_monthly_betas['Betas'] = df_monthly_betas['Betas'].apply(list)

# Set the Date column as the index
df_monthly_betas.set_index('Date', inplace=True)

# Convert each item in the 'Betas' list to a separate column
df_monthly_betas = df_monthly_betas['Betas'].apply(pd.Series)

# Set the index of df_monthly_betas to match df_48ind
df_monthly_betas.index = df_48ind.index

# Set the column names of df_monthly_betas to match df_48ind
df_monthly_betas.columns = df_48ind.columns

def calculate_idiosyncratic_volatility(start_date, end_date, df_daily_ret_reshaped, df_FF_daily):
    # Sélectionner les données pour la plage de dates spécifiée
    df_daily_ret_selected = df_daily_ret_reshaped.loc[start_date:end_date]
    df_FF_daily_selected = df_FF_daily.loc[start_date:end_date]

    # Maintenant, vous pouvez utiliser df_daily_ret_selected et df_FF_daily_selected pour votre régression
    Rft = df_FF_daily_selected['RF']
    Rm_t = df_FF_daily_selected['Mkt-RF']
    SMB = df_FF_daily_selected['SMB']
    HML = df_FF_daily_selected['HML']

    volatilities = []  # Create an empty list to store the volatilities

    for column in df_daily_ret_selected.columns:
        Ri_t = df_daily_ret_selected[column]

        Y = Ri_t - Rft
        X = pd.concat([Rm_t, SMB, HML], axis=1)

        X = sm.add_constant(X)

        model = sm.OLS(Y, X)
        results = model.fit()

        residuals = results.resid
        volatility = np.std(residuals)

        volatilities.append(volatility)  # Add the volatility to the list

    # Convert the list of volatilities to a numpy array
    volatilities = np.array(volatilities)

    return volatilities

# Get the first and last date in the data
first_date = df_daily_ret_reshaped.index.min()
last_date = df_daily_ret_reshaped.index.max()

# Create a date range for each month in the data
date_range = pd.date_range(start=first_date, end=last_date, freq='M')

# Calculate the volatilities for each month
monthly_volatilities = {}
for date in date_range:
    start_date = date  - pd.DateOffset(months=1) + pd.DateOffset(days=1)
    end_date = date
    volatilities = calculate_idiosyncratic_volatility(start_date, end_date, df_daily_ret, df_FF_daily)
    monthly_volatilities[date] = volatilities
    
        # Convert the dictionary to a DataFrame
df_monthly_vol = pd.DataFrame(monthly_volatilities.items(), columns=['Date', 'Betas'])

# Convert the Betas column from numpy arrays to lists
df_monthly_vol['Betas'] = df_monthly_vol['Betas'].apply(list)

# Set the Date column as the index
df_monthly_vol.set_index('Date', inplace=True)

# Convert each item in the 'Betas' list to a separate column
df_monthly_vol = df_monthly_vol['Betas'].apply(pd.Series)

# Set the index of df_monthly_vol to match df_48ind
df_monthly_vol.index = df_48ind.index

# Set the column names of df_monthly_vol to match df_48ind
df_monthly_vol.columns = df_48ind.columns

def select_extreme_values(row, num_values=5):
    sorted_row = row.sort_values(ascending=False)
    top_values = sorted_row.head(num_values)
    bottom_values = sorted_row.tail(num_values)
    return top_values, bottom_values

def get_returns(caracteristic, df_ret_shift, num_positions, weight_type='ew'):
    
    top_bottom_values = caracteristic.apply(select_extreme_values, axis=1)
    
    returns = []
    total_returns = []
    
    for date, values in top_bottom_values.items():
        top_indices, bottom_indices = values[0].index, values[1].index
        
        if weight_type == 'ew':
            weight = 1.0 / num_positions
            top_returns = df_ret_shift.loc[date, top_indices] * weight
            bottom_returns = df_ret_shift.loc[date, bottom_indices] * weight * -1  # short positions have negative weight
        elif weight_type == 'vw':
            top_values = df_ret_shift.loc[date, top_indices]
            bottom_values = df_ret_shift.loc[date, bottom_indices]
            top_weights = top_values.abs() / top_values.abs().sum()
            bottom_weights = bottom_values.abs() / bottom_values.abs().sum()
            top_returns = top_values * top_weights
            bottom_returns = bottom_values * bottom_weights * -1  # short positions have negative weight
        else:
            raise ValueError("weight_type must be either 'ew' or 'vw'")
        
        returns.append((top_returns.sum(), bottom_returns.sum()))
    
    total_returns = [sum(x) for x in returns]

    return total_returns

def get_sharpe_and_constants(caracteristic, df_ret_shift, num_positions, rf, df_FF_5, weight_type='ew'):
    total_returns = get_returns(caracteristic, df_ret_shift, num_positions, weight_type)

    # Convert list to pandas Series
    total_returns_series = pd.Series(total_returns)

    # Calculate Sharpe ratio
    sharpe_ratio = (total_returns_series.mean() - rf) / total_returns_series.std()

    # Convert list to pandas DataFrame
    total_returns_df = pd.DataFrame(total_returns, columns=['returns'])

    # Reset index
    total_returns_df = total_returns_df.reset_index(drop=True)
    df_FF_5 = df_FF_5.reset_index(drop=True)

    # Regression with 3, 4, and 5 factors
    constants = []
    for num_factors in [3, 4, 5]:
        factors = sm.add_constant(df_FF_5.iloc[:, :num_factors])
        model = sm.OLS(total_returns_df, factors)
        results = model.fit()
        constants.append(results.params['const'])

    return sharpe_ratio, constants

/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_3910/37480628.py:63: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_daily = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios_Daily.csv', header=5)


In [3]:
dataframes = [df_MC, df_BtoM, df_mom, df_monthly_betas, df_monthly_vol]
names = ['df_MC', 'df_BtoM', 'df_mom', 'df_monthly_betas', 'df_monthly_vol']
date_ranges = [('1970-06-01', '2024-01-01'), ('1990-01-01', '2024-01-01'), ('2000-01-01', '2024-01-01')]

for df, name in zip(dataframes, names):
    for start_date, end_date in date_ranges:
        # Filter the DataFrame for the given date range
        df_filtered = df.loc[start_date:end_date]
        df_FF_5_filtered = df_FF_5.loc[start_date:end_date]

        # Calculate Sharpe ratio and constants
        sharpe_ratio_ew, constants_ew = get_sharpe_and_constants(df_filtered, df_48ind, 5, rf, df_FF_5_filtered, 'ew')
        sharpe_ratio_vw, constants_vw = get_sharpe_and_constants(df_filtered, df_48ind, 5, rf, df_FF_5_filtered, 'vw')

        # Print results
        print(f"For {name} from {start_date} to {end_date}:")
        print("EW Sharpe Ratio: ", sharpe_ratio_ew)
        print("EW Constants: ", constants_ew)
        print("VW Sharpe Ratio: ", sharpe_ratio_vw)
        print("VW Constants: ", constants_vw)
        print("\n")

For df_MC from 1970-06-01 to 2024-01-01:
EW Sharpe Ratio:  -0.03724946826987043
EW Constants:  [0.1744608932148895, 0.36313018564047694, 0.41686805413367434]
VW Sharpe Ratio:  -0.07617980178009254
VW Constants:  [-0.1623462740384269, 0.09261705895242703, 0.1823155088663715]


For df_MC from 1990-01-01 to 2024-01-01:
EW Sharpe Ratio:  -0.02419187320336649
EW Constants:  [0.08498345574364308, 0.31321890666469904, 0.37834052903149046]
VW Sharpe Ratio:  -0.06279969361434276
VW Constants:  [-0.3105169510924926, 0.030599113423459178, 0.15201290942600637]


For df_MC from 2000-01-01 to 2024-01-01:
EW Sharpe Ratio:  -0.09314347733090564
EW Constants:  [-0.20212459364281926, 0.04962713675650925, 0.13339816783416092]
VW Sharpe Ratio:  -0.10691273080876101
VW Constants:  [-0.6129138022701306, -0.19224205940097078, -0.028036689082665513]


For df_BtoM from 1970-06-01 to 2024-01-01:
EW Sharpe Ratio:  -0.0903042430160105
EW Constants:  [-0.8677918194689576, -0.8073048677898551, -0.725928504649743]
V